In [1]:
import os
import shutil
import subprocess
import re
import operator
import pandas as pd
import pickle
from matplotlib import pyplot as plt

from xml.dom.minidom import Element
from androguard.core.bytecodes import apk
from androguard.core.bytecodes.dvm import DalvikVMFormat
from androguard.core.analysis import analysis
from androguard.decompiler.decompiler import DecompilerJADX
from androguard.core.androconf import show_logging

import numpy as np


In [ ]:
path = '/Users/apple/Desktop/ml/crawler/benign/air.com.and.games505.gemsofwar_v3.4.016-3400016_Android-4.1.apk'

In [ ]:
app = apk.APK(path)
app_dex = DalvikVMFormat(app.get_dex())

app_x = analysis.Analysis(app_dex)


cs = [cc.get_name() for cc in app_dex.get_classes()]


print cs

In [ ]:
for method in app_dex.get_methods():
    g = app_x.get_method(method)
    for ix in g.get_basic_blocks().get():
        for jx in ix.get_instructions():
            output = jx.get_output()
            match = re.search(r'(L[^;]*;)->[^\(]*\([^\)]*\).*', output)
            print match

In [ ]:
    if method.get_code() == None:
        continue
  

    # print "***********"
    # print "method beeing investigated - ", g


    for i in g.get_basic_blocks().get():

In [ ]:
external_api_dict = np.load('./api_calls_malware.npy')

In [ ]:
external_api_dict = list(set(external_api_dict.tolist()))

In [ ]:
(external_api_dict[:10])

In [ ]:
app = apk.APK(path)
app_dex =DalvikVMFormat(app.get_dex())
seq_no = 0
app_x = analysis.Analysis(app_dex)
cs = [cc.get_name() for cc in app_dex.get_classes()]
print len(app_dex.get_methods())
for method in app_dex.get_methods():
    g = app_x.get_method(method)


    if method.get_code() == None:
        continue


    print "***********"
    print "method beeing investigated - ", g


    for i in g.get_basic_blocks().get():
        print "i.childs : " ,i.childs

    if(i.childs!=[] and seq_no<50):

        call_no = 0
        for ins in i.get_instructions():
            output = ins.get_output()
            match = re.search(r'(L[^;]*;)->[^\(]*\([^\)]*\).*', output)
            if match and match.group(1) not in cs:

#                 print "instruction : ", ins.get_basic_blocks()
                print "output : ", output
                print "external api detected: ", match.group()


#                 if(i.childs!=[]):
#                     print "-------->",i.childs[0][2].childs
#                     break
                start_index = match.group().find('>')
                end_index = match.group().find('(')
                start_index+=1
                match_new = match.group()[start_index:end_index] + '()'
                
                print '+++++++++++++' ,match_new
                print call_no,external_api_dict.index(match_new),seq_no,'++++++++++++++'
        #       feature_vector[call_no,external_api_dict.index(match_new),seq_no] = 1
                call_no+=1
        rand_child_selected = np.random.randint(len(i.childs))
        print rand_child_selected

#         traverse_graph(i.childs[rand_child_selected][2],feature_vector,cs,call_no,seq_no,external_api_dict)
        seq_no+=1



In [ ]:
max_h = 50
max_calls = 50

In [ ]:
def get_compressed_feature_vector(path, external_api_dict):

    feature_vector = np.zeros((max_calls,max_h),dtype=int)
    call_no = 0
    seq_no = 0

    if path.endswith('.apk'):
        app = apk.APK(path)
        app_dex = DalvikVMFormat(app.get_dex())
    else: 
        app_dex = DalvikVMFormat(open(path, "rb").read())

    app_x = analysis.Analysis(app_dex)


    cs = [cc.get_name() for cc in app_dex.get_classes()]
    for method in app_dex.get_methods():
        g = app_x.get_method(method)

        if method.get_code() == None:
            continue

        for i in g.get_basic_blocks().get():
            if(i.childs!=[] and seq_no<max_h):
                call_no = 0
                for ins in i.get_instructions():
                    output = ins.get_output()
                    match = re.search(r'(L[^;]*;)->[^\(]*\([^\)]*\).*', output)
                    if match and match.group(1) not in cs:
                        start_index = match.group().find('>')
                        end_index = match.group().find('(')
                        start_index+=1
                        match_new = match.group()[start_index:end_index] + '()'

                        print '+++++++++++++' ,match_new
                        print call_no,seq_no,external_api_dict.index(match_new),'++++++++++++++'


                        feature_vector[call_no,seq_no] = external_api_dict.index(match_new)
                        call_no+=1
     

        
                rand_child_selected = np.random.randint(len(i.childs))

                traverse_graph(i.childs[rand_child_selected][2],feature_vector,cs,call_no,seq_no,external_api_dict)
                seq_no+=1

    return feature_vector


def traverse_graph(node,feature_vector,cs,call_no,seq_no,external_api_dict):
    print '~~~~~~~~~',node,'~~~~~~~~~~~~~'
    for ins in node.get_instructions():


        output = ins.get_output()
        match = re.search(r'(L[^;]*;)->[^\(]*\([^\)]*\).*', output)
        if match and match.group(1) not in cs:
            start_index = match.group().find('>')
            end_index = match.group().find('(')
            start_index+=1
            match_new = match.group()[start_index:end_index] + '()'


#         if match and match.group(1) not in cs and call_no<max_calls:

            feature_vector[call_no,seq_no] = external_api_dict.index(match_new)
            call_no+=1


  
    if(call_no<max_calls and node.childs!=[]):

        rand_child_selected = np.random.randint(len(node.childs))
        traverse_graph(node.childs[rand_child_selected][2],feature_vector,cs,call_no,seq_no,external_api_dict)


In [ ]:
get_compressed_feature_vector(path,external_api_dict)

In [ ]:
in_file = open('/Users/apple/Desktop/ml/Deep-Android-Malware-Detection/opcodeseq_creator/apk_test/AndroidManifest.xml', "r") # opening for [r]eading as [b]inary

in_file_2 = open('/Users/apple/Desktop/ml/apks/0a94f8da4ece3efdda467205b294f5338323604431dc41a66c68c833aba59263','r')

in_file_3 = open('/Users/apple/Desktop/ml/crawler/benign/ch.smalltech.ledflashlight.free_v1.95.05-Google-Play-19505_Android-4.0.3.apk','r')

In [ ]:
b= np.fromfile(in_file_3, dtype=np.uint8)

ns = np.ceil(np.sqrt(b.shape[0])).astype(int)
s = np.zeros(ns**2)
s[:b.size] = b
s = s.reshape(ns,ns)

print s.shape

In [ ]:
plt.imshow(s, cmap='gray')
plt.show()

In [2]:
X = []
y=[]

In [3]:
def get_images(folder_path,label):
        apk_folder = os.listdir(folder_path)
        print (folder_path)
        for ix in range(100):
            if apk_folder[ix]=='.DS_Store':
                    continue
            file_path = os.path.join(folder_path,apk_folder[ix])
            print 'Processing file ',ix,' ----> ',file_path
            in_file = open(file_path,'r')
            print 'Read file.........'
            binary_file = np.fromfile(in_file, dtype=np.uint8)
            img_shape = np.ceil(np.sqrt(binary_file.shape[0])).astype(int)
            if img_shape<224:
                img_shape= 224
            img = np.zeros(img_shape**2)
            img[:binary_file.size] = binary_file
            img = img.reshape(img_shape,img_shape)
            img = img[:224,:224]
            X.append(img)
            y.append(label)
            
#             plt.figure(ix/10)
#             plt.subplot(1,10,(ix%10)+1)
#             plt.imshow(img,cmap='gray')
#             plt.axis('off')
        return

In [ ]:
folder_path = '/Users/apple/Desktop/ml/crawler/benign'
get_images(folder_path,1)
np.save('./raw_apk_images_X.npy',np.array(X))
np.save('./raw_apk_images_y.npy',np.array(y))

/Users/apple/Desktop/ml/crawler/benign
Processing file  0  ---->  /Users/apple/Desktop/ml/crawler/benign/com.myfitnesspal.android_v6.37.2-10948_Android-5.0.apk
Read file.........
Processing file  1  ---->  /Users/apple/Desktop/ml/crawler/benign/com.rhmsoft.fm_v2.7.7-20770706_Android-4.0.apk
Read file.........
Processing file  2  ---->  /Users/apple/Desktop/ml/crawler/benign/com.jio.join_v5.0.6.multireg-5000604_Android-4.1.apk
Read file.........
Processing file  3  ---->  /Users/apple/Desktop/ml/crawler/benign/org.freeandroidtools.root_checker_v3.1.1-2293_Android-4.0.apk
Read file.........
Processing file  4  ---->  /Users/apple/Desktop/ml/crawler/benign/com.lookout_v10.22-5091e54-10220069_Android-4.1.apk
Read file.........
Processing file  5  ---->  /Users/apple/Desktop/ml/crawler/benign/com.urbandroid.lux_v8.3-275_Android-4.0.apk
Read file.........
Processing file  6  ---->  /Users/apple/Desktop/ml/crawler/benign/com.naturalmotion.csrracing_v5.0.1-21363_Android-4.0.3.apk
Read file....

In [ ]:
folder_path = '/Users/apple/Desktop/ml/apks/'
get_images(folder_path,0)
np.save('./raw_malware_images_X.npy',np.array(X))
np.save('./raw_malware_images_y.npy',np.array(y))

In [ ]:
def vgg(weight_path=None):
    model = Sequential()
    model.add(Conv2D(96,kernel_size=(9,9),activation='relu',padding='same',input_shape=(1,64,64)))
    model.add(MaxPooling2D((2,2)))
    
    model.add(Conv2D(192,kernel_size=(5,5),activation='relu',padding='same'))
    model.add(MaxPooling2D((2,2)))
    
    model.add(Conv2D(256,kernel_size=(5,5),activation='relu',padding='same'))
    model.add(MaxPooling2D((2,2)))
    
    model.add(Conv2D(384,kernel_size=(4,4),activation='relu',padding='same'))
    model.add(MaxPooling2D((2,2)))

    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(1024, activation='relu'))

    model.add(Dropout(0.5))
    model.add(Dense(241, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])
    
    model.summary()
    if weight_path:
        model.load_weights(weight_path)
    
    return model

In [ ]:
def get_images_2(folder_path):
        apk_folder = os.listdir(folder_path)
        print (folder_path)
        for ix in range(40):
            if apk_folder[ix]=='.DS_Store':
                    continue
            file_path = os.path.join(folder_path,apk_folder[ix])
            in_file = open(file_path,'r')
            binary_file = np.fromfile(in_file, dtype=np.uint8)
            img_shape = np.ceil(np.sqrt(binary_file.shape[0])).astype(int)
            img = np.zeros(img_shape**2)
            img[:binary_file.size] = binary_file
            img = img.reshape(img_shape,img_shape)
            plt.imshow(img, cmap='gray')
            plt.show()
        return

In [ ]:
from collections import defaultdict
import os
import operator
import json

print "Generating Features Dictionary"

rec = defaultdict(int)
OPSEQ_LEN = int(os.environ.get("OPSEQ_LEN", 10))

TRAININGSET_PATH = '/Users/apple/Desktop/ml/Deep-Android-Malware-Detection/opcodeseq_creator/opseq/'

for filename in os.listdir(TRAININGSET_PATH):
    if not filename.endswith("opseq"):
        continue
    filename = os.path.join(TRAININGSET_PATH, filename)
    print "processing %s" % filename
    with open(filename, 'r') as fr:
        lines = fr.readlines()
        tmp = []
        for line in lines:
            tmp.append(line[:-1])
        file_str = ''.join(tmp)

        for i in range(0, len(file_str) - OPSEQ_LEN):
            rec[file_str[i:i + OPSEQ_LEN]] += 1
sorted_rec = list(reversed(sorted(rec.items(), key=operator.itemgetter(1))))

with open("feature_dict.txt", "w") as fw:
    features = [x[0] for x in sorted_rec[:100]]
    json.dump(features, fw)

print "Done"